In [1]:
!pip install beautifulsoup4 
!pip install google
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [2]:
import data
import csv
import pandas as pd


In [3]:
def readInFile(filename = 'master.csv'):
  aup_data = []
  aup_id = []
  with open(filename) as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      line_count = 0
      for row in csv_reader:
          if line_count == 0:
              print(f'Column names are {", ".join(row)}')
              line_count += 1
          else:      
              aup_data.append(row[2])
              aup_id.append(row[1])

  return aup_data, aup_id

# Url Collection

In [6]:
# for each url we find their snapshots for the past 10 years, one for each year
def collectURL(aup_data):
  '''
  url_snapshots_dict: {
      url: (year, snapshot)
  }
  '''
  url_snapshots_dict = {}

  for url in aup_data:
    try:
      snapshots, timestamps = data.get_snapshots(url)
      snapshot_within_10_years = []
      listed_years = [] # a list that keeps the recorded years since we only want 1 snapshot each year
      for i in range(len(timestamps)):
        year = int(timestamps[i][:4])
        if year in listed_years:
          continue
        # if the timestamp is within 10 years, append corresponding snapshot
        if year >= 2013 and year <= 2023:
          snapshot_within_10_years.append((year, snapshots[i]))
          listed_years.append(year)
      url_snapshots_dict[url] = snapshot_within_10_years
    except Exception as inst: 
      print(f"Exception when dealing with {url}")
      print(inst)
      continue
  
  return url_snapshots_dict

In [7]:
def writeToCSV(url_snapshots_dict):
  try:
    df = pd.DataFrame(url_snapshots_dict.items(), columns=['aup-url', 'snapshots'])
    df.to_csv('snapshots_from_googlesearch.csv', index=False, header=True)
    return True
  except:
    print("Error wrting to CSV")
    return False

In [ ]:
aup_data = readInFile()
url_snapshots_dict = collectURL(aup_data)
writeToCSV(url_snapshots_dict)

In [13]:
aup_data, aup_id = readInFile()
print(len(aup_data))
print(len(aup_id))

Column names are source, id, url, filepath, link-to-filepath, sector
73
73


In [14]:
url_snapshots_dict = collectURL(aup_data)
len(url_snapshots_dict)

73

# Scrap AUP content

In [21]:
len(url_snapshots_dict)

73

In [33]:
# Now, we start to get AUP content using the urls
def scrapAUP(url_snapshots_dict, aup_id):
  idx = 0

  for url, tuples in url_snapshots_dict.items():
    print(f"URL currently working with {aup_id[idx]}")
    
    if len(tuples) == 0:
      print(f"{aup_id[idx]} is skipped")
      continue
    years = [t[0] for t in tuples]
    snapshots = [t[1] for t in tuples]
    for i in range(len(snapshots)):
      fname = f"snapshots/{aup_id[idx]}-{years[i]}.txt"
      data.get_aup_content(snapshots[i], fname)

    print(f"Finish with {aup_id[idx]}")
    idx += 1


In [32]:
scrapAUP(url_snapshots_dict, aup_id)

URL currently working with 14
Finish with 14
URL currently working with 40
Finish with 40
URL currently working with 43
Finish with 43
URL currently working with 156
Finish with 156
URL currently working with 301
URL currently working with 301
Finish with 301
URL currently working with 354
URL currently working with 354
Finish with 354
URL currently working with 585
URL currently working with 585
Finish with 585
URL currently working with 586
Finish with 586
URL currently working with 838
Finish with 838
URL currently working with 1028
URL currently working with 1028
Finish with 1028
URL currently working with 1091
Finish with 1091
URL currently working with 1145
URL currently working with 1145
Finish with 1145
URL currently working with 1296
Finish with 1296
URL currently working with 1350
Finish with 1350
URL currently working with 1657
Finish with 1657
URL currently working with 1706
URL currently working with 1706
Finish with 1706
URL currently working with 1710
Finish with 1710
UR

In [34]:
!tar -zcvf snapshots.tar.gz snapshots/ # get the data off colab

snapshots/
snapshots/17-2020.txt
snapshots/301-2021.txt
snapshots/129-2022.txt
snapshots/60-2021.txt
snapshots/7081-2022.txt
snapshots/1893-2023.txt
snapshots/1296-2014.txt
snapshots/156-2021.txt
snapshots/586-2023.txt
snapshots/22-2021.txt
snapshots/7705-2021.txt
snapshots/3352-2018.txt
snapshots/7287-2021.txt
snapshots/1091-2018.txt
snapshots/2661-2014.txt
snapshots/6215-2021.txt
snapshots/1091-2017.txt
snapshots/40-2023.txt
snapshots/1296-2023.txt
snapshots/9999-2022.txt
snapshots/1296-2017.txt
snapshots/1657-2017.txt
snapshots/3352-2021.txt
snapshots/2553-2023.txt
snapshots/50-2019.txt
snapshots/1296-2021.txt
snapshots/9864-2021.txt
snapshots/22-2018.txt
snapshots/40-2022.txt
snapshots/3041-2021.txt
snapshots/127-2020.txt
snapshots/2510-2017.txt
snapshots/1893-2020.txt
snapshots/9999-2023.txt
snapshots/57-2019.txt
snapshots/43-2020.txt
snapshots/2661-2021.txt
snapshots/586-2020.txt
snapshots/1657-2021.txt
snapshots/13-2022.txt
snapshots/7289-2020.txt
snapshots/14-2017.txt
snapshots

In [26]:
!rm -rf snapshots/